In [1]:
import numpy as np
import pandas as pd
from matplotlib import gridspec
from astropy.io import fits
from spec_tools import Source_present
from spec_exam import Gen_beam_fits
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
from glob import glob
import seaborn as sea
import os
from grizli import model as griz_model

from time import time
sea.set(style='white')
sea.set(style='ticks')
sea.set_style({'xtick.direct'
               'ion': 'in','xtick.top':True,'xtick.minor.visible': True,
               'ytick.direction': "in",'ytick.right': True,'ytick.minor.visible': True})
cmap = sea.cubehelix_palette(12, start=2, rot=.2, dark=0, light=1.0, as_cmap=True)
%config InlineBackend.figure_format = 'retina'
%matplotlib inline

### set home for files
hpath = os.environ['HOME'] + '/'

In [2]:
gal_id = 41520 

In [3]:
filelist = glob(hpath + 'Clear_data/flt_files/*flt.fits')

In [4]:
RA,DEC = (53.152726, -27.716251)

flt_files = []
obj_pos =[]
for i in range(len(filelist)):
    pres,pos=Source_present(filelist[i],RA,DEC)
    if pres==True:
        obj_pos.append(pos)
        flt_files.append(filelist[i])

False IMAGING
False SPECTROSCOPIC
False SPECTROSCOPIC
False IMAGING
False SPECTROSCOPIC
False SPECTROSCOPIC
False IMAGING
False SPECTROSCOPIC
False SPECTROSCOPIC
False IMAGING
False SPECTROSCOPIC
False SPECTROSCOPIC
False SPECTROSCOPIC
False SPECTROSCOPIC
False IMAGING
False SPECTROSCOPIC
False SPECTROSCOPIC
False IMAGING
False SPECTROSCOPIC
False SPECTROSCOPIC
False IMAGING
False SPECTROSCOPIC
False SPECTROSCOPIC
False IMAGING
False IMAGING
False SPECTROSCOPIC
False SPECTROSCOPIC
False IMAGING
False SPECTROSCOPIC
False SPECTROSCOPIC
False IMAGING
False SPECTROSCOPIC
False SPECTROSCOPIC
False IMAGING
False SPECTROSCOPIC
False SPECTROSCOPIC
False IMAGING
False SPECTROSCOPIC
False SPECTROSCOPIC
False IMAGING
False SPECTROSCOPIC
False SPECTROSCOPIC
False IMAGING
False SPECTROSCOPIC
False SPECTROSCOPIC
False IMAGING
False SPECTROSCOPIC
False SPECTROSCOPIC
False IMAGING
False SPECTROSCOPIC
False SPECTROSCOPIC
False IMAGING
False SPECTROSCOPIC
False SPECTROSCOPIC
False IMAGING
False SPECTROS

In [12]:
orients = []
for i in range(len(flt_files)):
    dat = fits.open(flt_files[i])
    orients.append(np.round(dat[0].header['PA_V3']).astype(int))

In [14]:
xpos,ypos = np.array(obj_pos).T

In [20]:
obj_DB = pd.DataFrame({'file' : flt_files, 'orient' : orients, 'xpos' : xpos, 'ypos' : ypos})

In [22]:
obj_DB.ra = 53.152726
obj_DB.dec = -27.716251

In [23]:
obj_DB.ra

53.152726

In [3]:
obj_DB= pd.read_pickle('../dataframes/file_list/{0}.pkl'.format(gal_id))

In [4]:
num = 1
pa = obj_DB.orient[0]

for i in obj_DB.index:
    if pa  == obj_DB.orient[i]:
        
        Gen_beam_fits(hpath + 'Clear_data/goodss_mosaic/goodss_3dhst.v4.0.F125W_orig_sci.fits',
         hpath + 'Clear_data/goodss_mosaic/goodss_3dhst.v4.0.F160W_seg.fits',
         obj_DB.file[i],
         hpath + 'Clear_data/goodss_mosaic/goodss_3dhst.v4.3.cat',gal_id,num)
        
        pa  = obj_DB.orient[i]
        num += 1
    
    else:
        pa  = obj_DB.orient[i]
        num = 1
        
        Gen_beam_fits(hpath + 'Clear_data/goodss_mosaic/goodss_3dhst.v4.0.F125W_orig_sci.fits',
         hpath + 'Clear_data/goodss_mosaic/goodss_3dhst.v4.0.F160W_seg.fits',
         obj_DB.file[i],
         hpath + 'Clear_data/goodss_mosaic/goodss_3dhst.v4.3.cat',gal_id,num)
        
        num+=1



/Users/Vince.ec/Clear_data/flt_files/icxt30s0q_flt.fits / blot reference /Users/Vince.ec/Clear_data/goodss_mosaic/goodss_3dhst.v4.0.F125W_orig_sci.fits[0]
The following task in the stsci.skypac package can be run with TEAL:
                                    skymatch                                    
The following tasks in the drizzlepac package can be run with TEAL:
    astrodrizzle       imagefindpars           mapreg              photeq       
     pixreplace           pixtopix            pixtosky        refimagefindpars  
     resetbits          runastrodriz          skytopix           tweakback      
      tweakreg           updatenpol
Using default C-based coordinate transformation...
/Users/Vince.ec/Clear_data/flt_files/icxt30s0q_flt.fits / blot segmentation /Users/Vince.ec/Clear_data/goodss_mosaic/goodss_3dhst.v4.0.F160W_seg.fits[0]
Using default C-based coordinate transformation...
Using default C-based coordinate transformation...
/Users/Vince.ec/Clear_data/flt_files/icxt3

In [5]:
obj_DB.file

0     /Users/Vince.ec/Clear_data/flt_files/icxt30s0q...
1     /Users/Vince.ec/Clear_data/flt_files/icxt30s2q...
2     /Users/Vince.ec/Clear_data/flt_files/icxt30s5q...
3     /Users/Vince.ec/Clear_data/flt_files/icxt30s7q...
4     /Users/Vince.ec/Clear_data/flt_files/icxt31r1q...
5     /Users/Vince.ec/Clear_data/flt_files/icxt31r3q...
6     /Users/Vince.ec/Clear_data/flt_files/icxt31r6q...
7     /Users/Vince.ec/Clear_data/flt_files/icxt31r8q...
8     /Users/Vince.ec/Clear_data/flt_files/icxt32nlq...
9     /Users/Vince.ec/Clear_data/flt_files/icxt32nmq...
10    /Users/Vince.ec/Clear_data/flt_files/icxt32npq...
11    /Users/Vince.ec/Clear_data/flt_files/icxt32nrq...
12    /Users/Vince.ec/Clear_data/flt_files/icxt33roq...
13    /Users/Vince.ec/Clear_data/flt_files/icxt33rpq...
14    /Users/Vince.ec/Clear_data/flt_files/icxt33rsq...
15    /Users/Vince.ec/Clear_data/flt_files/icxt33ruq...
16    /Users/Vince.ec/Clear_data/flt_files/icxt34ekq...
17    /Users/Vince.ec/Clear_data/flt_files/icxt3